<a href="https://colab.research.google.com/github/DanPhala/Python-Prod/blob/main/LiveTec_assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings("ignore")



# New Section

In [ ]:
!pip install pandas openpyxl


In [8]:
import pandas as pd
import json
from datetime import datetime, timedelta
import os
from google.colab import files


In [ ]:

uploaded = files.upload()

file_name = list(uploaded.keys())[0]
file_path = os.path.abspath(file_name)

In [13]:
df = pd.read_csv(file_path, encoding='unicode_escape')
print(df)

               Week               District          County  \
0     Week 1 (2023)         Aberdeenshire   Aberdeenshire    
1     Week 1 (2023)                  Angus           Angus   
2     Week 1 (2023)                   Arun     West Sussex   
3     Week 1 (2023)  Basingstoke and Deane       Hampshire   
4     Week 1 (2023)  Basingstoke and Deane       Hampshire   
..              ...                    ...             ...   
433  Week 36 (2023)                   Fife            Fife   
434  Week 36 (2023)               Highland        Highland   
435  Week 36 (2023)                Lothian         Lothian   
436  Week 36 (2023)               Somerset        Somerset   
437  Week 37 (2023)                  Angus           Angus   

                       Species (Positive)                    Species (Latin)  \
0    Mute Swan (1), Pink Footed Goose (2)  Cygnus olor, Anser brachyrhynchus   
1                   Pink Footed Goose (3)               Anser brachyrhynchus   
2              

#weekly + Monthly


In [15]:

def parse_date(input_str):
    if "Week" in input_str:
        week_number = int(input_str.split()[1])
        year = int(input_str.split()[-1][1:-1])

        start_of_week = pd.to_datetime(f'{year}-W{week_number}-1', format='%Y-W%U-%w')

        end_of_week = pd.to_datetime(f'{year}-W{week_number + 1}-0', format='%Y-W%U-%w') - pd.Timedelta(seconds=1)

        return start_of_week, end_of_week
    else:
        date = pd.to_datetime(input_str, format='%B %Y', errors='coerce')
        if pd.isna(date):
            return "Invalid date format."


        start_of_month = date.replace(day=1)
        end_of_month = (date + pd.DateOffset(months=1) - pd.Timedelta(days=1)).replace(hour=23, minute=59, second=59)

        return start_of_month, end_of_month

df = pd.read_csv(file_path, encoding='unicode_escape')

result_df = pd.DataFrame(columns=['title', 'district', 'country', 'numberOfFindings','totalNumberofBirdsTestingPositive', 'species','periodStart', 'periodEnd'])
monthly_result_df = pd.DataFrame(columns=['title', 'district', 'country', 'numberOfFindings','totalNumberofBirdsTestingPositive', 'species','periodStart', 'periodEnd'])

for index, row in df.iterrows():
    week_title = f"{row['Week']}"
    district = row['District']
    country = row['County']
    species_name = row['Species (Positive)']
    latin_name = row['Species (Latin)']
    pathotype = row['Pathotype']
    total = row['Total']
    week_start, week_end = parse_date(week_title)

    existing_row = result_df[(result_df['title'] == week_title) & (result_df['district'] == district)]
    if not existing_row.empty:
        existing_row_index = existing_row.index[0]
        result_df.at[existing_row_index, 'numberOfFindings'] += 1
        result_df.at[existing_row_index, 'totalNumberofBirdsTestingPositive'] += total
        existing_species = next((s for s in result_df.at[existing_row_index, 'species'] if s['latinName'] == latin_name), None)
        if existing_species:
            existing_species['total'] += total
        else:
            result_df.at[existing_row_index, 'species'].append({
                'name': re.sub(r'\(\d+\)', '', species_name.replace('\u00a0', ' ')),
                'latinName': latin_name,
                'pathotype': pathotype,
                'total': total
            })
    else:
        result_df = result_df._append({
            'title': week_title,
            'district': district.replace('\u00a0', ' '),
            'country': country.replace('\u00a0', ' '),
            'numberOfFindings': 1,
            'totalNumberofBirdsTestingPositive': total,
            'species': [{
                'name':  re.sub(r'\(\d+\)', '', species_name.replace('\u00a0', ' ')),
                'latinName': str(latin_name).replace('\u00a0', ' '),
                'pathotype': pathotype.replace('\u00a0', ' '),
                'total': total
            }],
            'periodStart': week_start.isoformat(),
            'periodEnd':  week_end.isoformat()
        }, ignore_index=True)


    month_title = week_start.strftime('%B %Y')
    existing_monthly_row = monthly_result_df[(monthly_result_df['title'] == month_title) & (monthly_result_df['district'] == district)]
    if not existing_monthly_row.empty:
        existing_monthly_index = existing_monthly_row.index[0]
        monthly_result_df.at[existing_monthly_index, 'numberOfFindings'] += 1
        monthly_result_df.at[existing_monthly_index, 'totalNumberofBirdsTestingPositive'] += total
        existing_species = next((s for s in monthly_result_df.at[existing_monthly_index, 'species'] if s['latinName'] == latin_name), None)
        if existing_species:
            existing_species['total'] += total
        else:
            monthly_result_df.at[existing_monthly_index, 'species'].append({
                'name': re.sub(r'\(\d+\)', '', species_name.replace('\u00a0', ' ')),
                'latinName': latin_name,
                'pathotype': pathotype,
                'total': total
            })
    else:
        monthly_result_df = monthly_result_df._append({
            'title': month_title,
            'district': district.replace('\u00a0', ' '),
            'country': country.replace('\u00a0', ' '),
            'numberOfFindings': 1,
            'totalNumberofBirdsTestingPositive': total,
            'species': [{
                'name':  re.sub(r'\(\d+\)', '', species_name.replace('\u00a0', ' ')),
                'latinName': str(latin_name).replace('\u00a0', ' '),
                'pathotype': pathotype.replace('\u00a0', ' '),
                'total': total
            }],
            'periodStart': week_start.replace(day=1).isoformat(),
            'periodEnd': (week_start.replace(day=1) + pd.offsets.MonthEnd()).isoformat()
        }, ignore_index=True)

json_result = result_df.to_json(orient='records', default_handler=str, indent=2)
json_monthly_result = monthly_result_df.to_json(orient='records', default_handler=str, indent=2)

json_file_name = os.path.splitext(file_name)[0] #+ '.json'

weekly_file_json_name = json_file_name + '-weekly.json'
monthly_file_json_name = json_file_name + '-monthly.json'

with open(weekly_file_json_name , 'w') as weekly_file:
    weekly_file.write(json_result)
    print(f"Weekly files done ...")
with open(monthly_file_json_name ,  'w') as monthly_file:
    monthly_file.write(json_monthly_result)
    print(f"Monthly files done ...")




Weekly files done ...
Monthly files done ...
